# Simple Agent Evaluator

This notebook provides a simplified agent evaluation framework extracted from the Agent Evaluation library. It supports multiple model providers through Amazon Bedrock including Anthropic Claude, Amazon Nova, Meta Llama, and OpenAI GPT models.

## Features
- Multi-provider support (Anthropic, Amazon, Meta, OpenAI)
- Cross-region inference endpoints
- On-demand endpoints
- Configurable AWS regions
- Automated conversation evaluation
- **Test file loading from current working directory**
- **✅ FIXED: Amazon Nova compatibility with correct inferenceConfig**

## Available Models
- **Anthropic**: `us.anthropic.claude-3-sonnet-20240229-v1:0`, `us.anthropic.claude-3-7-sonnet-20250219-v1:0`
- **Amazon Nova**: `amazon.nova-premier-v1:0`, `amazon.nova-pro-v1:0`, `amazon.nova-lite-v1:0`, `amazon.nova-micro-v1:0` ✅ **FIXED**
- **Meta Llama**: `us.meta.llama4-maverick-17b-instruct-v1:0`, `us.meta.llama3-2-90b-instruct-v1:0`
- **OpenAI**: `openai.gpt-oss-120b-1:0`, `openai.gpt-oss-20b-1:0`

## Usage
1. Place your test JSON file (e.g., `tests_structure.json`) in the same directory as this notebook
2. Configure your model, agent, and region settings in the final cell
3. Run all cells to execute the evaluation

## Recent Fix
- ✅ **Amazon Nova Final Fix**: Now uses correct `inferenceConfig` with `maxTokens` parameter
- ❌ **Previous attempts**: `max_new_tokens` and `max_tokens` were both incorrect
- ✅ **Current solution**: Uses proper Nova API format with nested configuration

## Install Required Dependencies

First, let's install all the required packages:

In [58]:
# Install required packages
!pip install boto3 click jinja2 jsonpath-ng markdown-it-py pydantic pyyaml rich

## Import Required Libraries

Import all necessary Python libraries for the agent evaluation framework:

In [59]:
#!/usr/bin/env python3
"""
Simplified Agent Evaluation Script
Extracted from the Agent Evaluation framework to provide core evaluation functionality.
"""

import json
import uuid
import boto3
import re
import os
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

## Model Provider Configuration

Define the supported model providers and configuration classes. This supports:
- **Anthropic**: Claude models via cross-region inference
- **Amazon**: Nova models via cross-region inference (✅ **FIXED** with `inferenceConfig`)
- **Meta**: Llama models via cross-region inference
- **OpenAI**: GPT models via on-demand endpoints

In [60]:
class ModelProvider(Enum):
    ANTHROPIC = "anthropic"
    AMAZON = "amazon"
    META = "meta"
    OPENAI = "openai"


@dataclass
class BedrockModelConfig:
    model_id: str
    request_body: Dict
    
    @property
    def provider(self) -> ModelProvider:
        if "anthropic" in self.model_id:
            return ModelProvider.ANTHROPIC
        elif "amazon" in self.model_id:
            return ModelProvider.AMAZON
        elif "meta" in self.model_id:
            return ModelProvider.META
        elif "openai" in self.model_id:
            return ModelProvider.OPENAI
        else:
            raise ValueError(f"Unsupported model ID: {self.model_id}")

## Conversation and Target Agent Classes

Define classes for:
- **Conversation**: Captures multi-turn interactions between user and agent
- **BedrockAgentTarget**: Handles communication with Amazon Bedrock agents

In [61]:
class Conversation:
    """Captures the interaction between a user and an agent."""
    
    def __init__(self):
        self.messages = []
        self.turns = 0

    def add_turn(self, user_message: str, agent_response: str):
        """Record a turn in the conversation."""
        self.messages.extend([("USER", user_message), ("AGENT", agent_response)])
        self.turns += 1

    def __iter__(self):
        return iter(self.messages)


class BedrockAgentTarget:
    """A target encapsulating an Amazon Bedrock agent."""
    
    def __init__(self, bedrock_agent_id: str, bedrock_agent_alias_id: str, aws_region: str = "us-east-1"):
        self.bedrock_agent_id = bedrock_agent_id
        self.bedrock_agent_alias_id = bedrock_agent_alias_id
        self.session_id = str(uuid.uuid4())
        self.client = boto3.client("bedrock-agent-runtime", region_name=aws_region)

    def invoke(self, prompt: str) -> str:
        """Invoke the target with a prompt."""
        response = self.client.invoke_agent(
            agentId=self.bedrock_agent_id,
            agentAliasId=self.bedrock_agent_alias_id,
            sessionId=self.session_id,
            inputText=prompt,
            enableTrace=True,
        )

        stream = response["completion"]
        completion = ""
        
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                completion += chunk.get("bytes").decode()

        return completion

## Bedrock Request Handler

This class handles the communication with different Bedrock model providers. It:
- **Builds request bodies** specific to each provider's API format
- **Parses responses** from different model providers
- **Supports all four providers**: Anthropic, Amazon, Meta, and OpenAI
- **✅ FIXED Amazon Nova**: Uses correct `inferenceConfig` structure

In [62]:
class BedrockRequestHandler:
    """Static class for building requests to and receiving requests from Bedrock."""
    
    @staticmethod
    def build_request_body(request_body: Dict, model_config: BedrockModelConfig, 
                          system_prompt: str, prompt: str) -> Dict:
        """Build request body for different model providers."""
        if model_config.provider == ModelProvider.ANTHROPIC:
            request_body["system"] = system_prompt
            if "messages" in request_body:
                request_body["messages"][0]["content"][0]["text"] = prompt
        elif model_config.provider == ModelProvider.AMAZON:
            # Amazon Nova models use system array format
            request_body["system"] = [{"text": system_prompt}]
            if "messages" in request_body:
                request_body["messages"][0]["content"][0]["text"] = prompt
        elif model_config.provider == ModelProvider.META:
            # Meta Llama models use prompt format
            request_body["prompt"] = (
                f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}"
                f"<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}"
                "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            )
        elif model_config.provider == ModelProvider.OPENAI:
            # OpenAI models use messages format similar to OpenAI API
            if "messages" in request_body:
                request_body["messages"] = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ]
        return request_body

    @staticmethod
    def parse_completion_from_response(response: Dict, model_config: BedrockModelConfig) -> str:
        """Parse completion from different model provider responses."""
        response_body = response.get("body").read()
        response_json = json.loads(response_body)
        
        if model_config.provider == ModelProvider.ANTHROPIC:
            completion = response_json["content"][0]["text"]
        elif model_config.provider == ModelProvider.AMAZON:
            # Amazon Nova models return output in message format
            completion = response_json["output"]["message"]["content"][0]["text"]
        elif model_config.provider == ModelProvider.META:
            # Meta Llama models return generation
            completion = response_json["generation"]
        elif model_config.provider == ModelProvider.OPENAI:
            # OpenAI models return choices with message content
            completion = response_json["choices"][0]["message"]["content"]
        else:
            raise ValueError(f"Unsupported provider: {model_config.provider}")
            
        return completion

## Simple Agent Evaluator Class

The main evaluator class that:
- **Initializes** the evaluator with model configuration
- **Creates model configs** for different providers (✅ **Nova FIXED** with `inferenceConfig`)
- **Generates responses** using the evaluator model
- **Evaluates conversations** and provides test results
- **Uses current working directory** for test file paths

In [63]:
class SimpleAgentEvaluator:
    """Simplified agent evaluator based on the canonical evaluator logic."""
    
    def __init__(self, evaluator_model: str, agent_id: str, agent_alias_id: str, 
                 aws_region: str = "us-east-1", max_turns: int = 10):
        self.evaluator_model = evaluator_model
        self.agent_id = agent_id
        self.agent_alias_id = agent_alias_id
        self.aws_region = aws_region
        self.max_turns = max_turns
        
        # Initialize Bedrock client for evaluator
        self.bedrock_client = boto3.client("bedrock-runtime", region_name=aws_region)
        
        # Initialize target agent
        self.target = BedrockAgentTarget(agent_id, agent_alias_id, aws_region)
        
        # Configure evaluator model based on provider
        self.model_config = self._create_model_config(evaluator_model)

    def _create_model_config(self, model_id: str) -> BedrockModelConfig:
        """Create model configuration based on the model provider."""
        if "anthropic" in model_id:
            return BedrockModelConfig(
                model_id=model_id,
                request_body={
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 4000,
                    "temperature": 0.0,
                    "messages": [{"role": "user", "content": [{"text": ""}]}]
                }
            )
        elif "amazon" in model_id:
            return BedrockModelConfig(
                model_id=model_id,
                request_body={
                    "inferenceConfig": {
                        "maxTokens": 4000,
                        "temperature": 0.0
                    },
                    "messages": [{"role": "user", "content": [{"text": ""}]}]
                }
            )
        elif "meta" in model_id:
            return BedrockModelConfig(
                model_id=model_id,
                request_body={
                    "max_gen_len": 4000,
                    "temperature": 0.0,
                    "prompt": ""
                }
            )
        elif "openai" in model_id:
            return BedrockModelConfig(
                model_id=model_id,
                request_body={
                    "max_tokens": 4000,
                    "temperature": 0.0,
                    "messages": []  # Will be populated by build_request_body
                }
            )
        else:
            raise ValueError(f"Unsupported model: {model_id}")

    def _extract_content_from_xml(self, xml_data: str, element_names: List[str]) -> Tuple:
        """Extract content from XML tags."""
        content = []
        for e in element_names:
            pattern = rf"<{e}>(.*?)</{e}>"
            match = re.search(pattern, xml_data, re.DOTALL)
            content.append(match.group(1).strip() if match else None)
        return tuple(content)

    def _generate(self, system_prompt: str, prompt: str, output_xml_element: str) -> Tuple[str, str]:
        """Generate response using the evaluator model."""
        request_body = BedrockRequestHandler.build_request_body(
            request_body=self.model_config.request_body.copy(),
            model_config=self.model_config,
            system_prompt=system_prompt,
            prompt=prompt,
        )

        response = self.bedrock_client.invoke_model(
            modelId=self.model_config.model_id, 
            body=json.dumps(request_body)
        )

        completion = BedrockRequestHandler.parse_completion_from_response(
            response=response,
            model_config=self.model_config
        )

        output, reasoning = self._extract_content_from_xml(
            completion, [output_xml_element, "thinking"]
        )

        return output, reasoning

    def _generate_initial_prompt(self, step: str) -> str:
        """Generate the initial prompt for the conversation."""
        system_prompt = """You are a quality assurance engineer testing a conversational agent.

Your job is to generate an initial prompt based on the provided step.

Please think hard about the response in <thinking> tags before providing only the initial prompt
within <initial_prompt> tags."""

        prompt = f"Generate an initial prompt for this step: {step}"
        
        initial_prompt, reasoning = self._generate(
            system_prompt=system_prompt,
            prompt=prompt,
            output_xml_element="initial_prompt",
        )
        
        return initial_prompt or step  # Fallback to original step if generation fails

    def _generate_test_status(self, steps: List[str], conversation: Conversation) -> str:
        """Generate test status to determine if all steps have been attempted."""
        system_prompt = """You are a quality assurance engineer evaluating a conversation between an USER and an AGENT.

Your job is to analyze the conversation in <conversation> tags and a list of steps in <steps> tags.

You will classify the conversation into the following categories:

- A: All steps have been attempted in the conversation.
- B: Not all steps have been attempted in the conversation.

Please think hard about the response in <thinking> tags before providing only the category letter
within <category> tags."""

        conversation_text = "\n".join([f"{sender}: {message}" for sender, message in conversation])
        steps_text = "\n".join([f"{i+1}. {step}" for i, step in enumerate(steps)])
        
        prompt = f"""Here are the steps and conversation:

<steps>
{steps_text}
</steps>

<conversation>
{conversation_text}
</conversation>"""

        test_status, reasoning = self._generate(
            system_prompt=system_prompt,
            prompt=prompt,
            output_xml_element="category",
        )
        
        return test_status

    def _generate_evaluation(self, expected_results: List[str], conversation: Conversation) -> Tuple[str, str]:
        """Generate evaluation of the conversation against expected results."""
        system_prompt = """You are a quality assurance engineer evaluating a conversation between an USER and an AGENT.

Your job is to analyze the conversation in <conversation> tags and a list of expected results
in <expected_results> tags.

You will classify the the conversation into the following categories:

- A: All of the expected results can be observed in the conversation.
- B: Not all of the expected results can be observed in the conversation.

Please think hard about the response in <thinking> tags before providing only the category letter
within <category> tags."""

        conversation_text = "\n".join([f"{sender}: {message}" for sender, message in conversation])
        expected_results_text = "\n".join([f"{i+1}. {result}" for i, result in enumerate(expected_results)])
        
        prompt = f"""Here are the expected results and conversation:

<expected_results>
{expected_results_text}
</expected_results>

<conversation>
{conversation_text}
</conversation>"""

        evaluation, reasoning = self._generate(
            system_prompt=system_prompt,
            prompt=prompt,
            output_xml_element="category",
        )
        
        return evaluation, reasoning

    def _generate_user_response(self, steps: List[str], conversation: Conversation) -> str:
        """Generate the next user response based on steps and conversation history."""
        system_prompt = """You are a quality assurance engineer testing a conversational agent.

Your job is to generate the next user response based on the provided steps and conversation history.

Please think hard about the response in <thinking> tags before providing only the user response
within <user_response> tags."""

        conversation_text = "\n".join([f"{sender}: {message}" for sender, message in conversation])
        steps_text = "\n".join([f"{i+1}. {step}" for i, step in enumerate(steps)])
        
        prompt = f"""Here are the steps and conversation history:

<steps>
{steps_text}
</steps>

<conversation>
{conversation_text}
</conversation>

Generate the next appropriate user response to continue the conversation."""

        user_response, reasoning = self._generate(
            system_prompt=system_prompt,
            prompt=prompt,
            output_xml_element="user_response",
        )
        
        return user_response

    def evaluate_test(self, test_name: str, questions: List[str], expected_results: List[str]) -> Dict:
        """Evaluate a single test with multiple questions."""
        conversation = Conversation()
        passed = False
        result = "Maximum turns reached."
        reasoning = ""
        
        print(f"\n=== Evaluating Test: {test_name} ===")
        
        while conversation.turns < self.max_turns:
            if conversation.turns == 0:
                # Start conversation with first question
                user_input = questions[0] if questions else "Hello"
                print(f"\nTurn {conversation.turns + 1}")
                print(f"USER: {user_input}")
            else:
                # Generate next user response
                user_input = self._generate_user_response(questions, conversation)
                print(f"\nTurn {conversation.turns + 1}")
                print(f"USER: {user_input}")

            # Get agent response
            agent_response = self.target.invoke(user_input)
            print(f"AGENT: {agent_response}")
            
            # Add turn to conversation
            conversation.add_turn(user_input, agent_response)

            # Check test status
            test_status = self._generate_test_status(questions, conversation)
            
            if test_status == "A":  # All steps attempted
                # Evaluate conversation
                eval_category, reasoning = self._generate_evaluation(expected_results, conversation)
                
                if eval_category == "A":
                    result = "All of the expected results can be observed in the conversation."
                    passed = True
                else:
                    result = "Not all of the expected results can be observed in the conversation."
                
                break

        return {
            "test_name": test_name,
            "passed": passed,
            "result": result,
            "reasoning": reasoning,
            "conversation": [(sender, message) for sender, message in conversation.messages],
            "turns": conversation.turns
        }

    def run_evaluation(self, tests_file: str) -> Dict:
        """Run evaluation on all tests from the JSON file."""
        # Get current working directory and construct full path
        current_dir = os.getcwd()
        tests_file_path = os.path.join(current_dir, tests_file)
        
        print(f"Loading tests from: {tests_file_path}")
        
        with open(tests_file_path, 'r') as f:
            tests_data = json.load(f)
        
        results = []
        total_tests = 0
        passed_tests = 0
        
        for test_name, test_data in tests_data.items():
            # Extract questions and expected results from the multi-turn structure
            questions = []
            expected_results = []
            
            for question_key in sorted(test_data.keys()):
                if question_key.startswith('question_'):
                    questions.append(test_data[question_key]['question'])
                    expected_results.append(test_data[question_key]['expected_results'])
            
            # Run evaluation
            test_result = self.evaluate_test(test_name, questions, expected_results)
            results.append(test_result)
            
            total_tests += 1
            if test_result['passed']:
                passed_tests += 1
        
        # Calculate pass rate
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        return {
            "pass_rate": f"{pass_rate:.1f}%",
            "total_tests": total_tests,
            "passed_tests": passed_tests,
            "results": results
        }

## Configuration and Main Execution

Configure the evaluation parameters and run the evaluation:
- **Model Selection**: Choose from Anthropic, Amazon, Meta, or OpenAI models
- **Agent Configuration**: Set your Bedrock agent ID and alias
- **Region Configuration**: Set your AWS region
- **Test File**: Loads from current working directory
- **Run Evaluation**: Execute the evaluation process


In [66]:
def main():
    """Main function to run the evaluation."""
    # Configuration - Update these values
    # Available model options:
    # - Anthropic: "us.anthropic.claude-3-sonnet-20240229-v1:0", "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    # - Amazon Nova: "amazon.nova-premier-v1:0", "amazon.nova-pro-v1:0", "amazon.nova-lite-v1:0", "amazon.nova-micro-v1:0"
    # - Meta Llama: "us.meta.llama4-maverick-17b-instruct-v1:0", "us.meta.llama3-2-90b-instruct-v1:0"
    # - OpenAI: "openai.gpt-oss-120b-1:0", "openai.gpt-oss-20b-1:0"
    
    EVALUATOR_MODEL = "us.meta.llama4-maverick-17b-instruct-v1:0"  # Amazon Nova Pro model
    AGENT_ID = "CARG5UXPD9"  # Replace with your actual agent ID
    AGENT_ALIAS_ID = "0RV9TBGQC4"  # Replace with your actual alias ID
    AWS_REGION = "us-east-1"  # Replace with your region
    TESTS_FILE = "tests_structure.json"
    
    # Initialize evaluator
    evaluator = SimpleAgentEvaluator(
        evaluator_model=EVALUATOR_MODEL,
        agent_id=AGENT_ID,
        agent_alias_id=AGENT_ALIAS_ID,
        aws_region=AWS_REGION,
        max_turns=10
    )
    
    # Run evaluation
    print("Starting Agent Evaluation...")
    print(f"Evaluator Model: {EVALUATOR_MODEL}")
    print(f"AWS Region: {AWS_REGION}")
    print(f"Target Agent ID: {AGENT_ID}")
    print(f"Target Agent Alias: {AGENT_ALIAS_ID}")
    
    evaluation_results = evaluator.run_evaluation(TESTS_FILE)
    
    # Print results
    print(f"\n{'='*60}")
    print("EVALUATION SUMMARY")
    print(f"{'='*60}")
    print(f"Pass Rate: {evaluation_results['pass_rate']}")
    print(f"Tests Passed: {evaluation_results['passed_tests']}/{evaluation_results['total_tests']}")
    
    print(f"\n{'='*60}")
    print("DETAILED RESULTS")
    print(f"{'='*60}")
    
    for result in evaluation_results['results']:
        print(f"\nTest: {result['test_name']}")
        print(f"Status: {'PASSED' if result['passed'] else 'FAILED'}")
        print(f"Result: {result['result']}")
        print(f"Reasoning: {result['reasoning']}")
        print(f"Turns: {result['turns']}")
        
        print("\nConversation:")
        for sender, message in result['conversation']:
            print(f"  {sender}: {message}")
        print("-" * 40)


if __name__ == "__main__":
    main()


Starting Agent Evaluation...
Evaluator Model: us.meta.llama4-maverick-17b-instruct-v1:0
AWS Region: us-east-1
Target Agent ID: CARG5UXPD9
Target Agent Alias: 0RV9TBGQC4
Loading tests from: /home/sagemaker-user/strands-langfuse/multi-agents-fmw/bedrock-agents-langfuse/py_br_agent_evaluator/tests_structure.json

=== Evaluating Test: simple_bitcoin_question ===

Turn 1
USER: what is the current bitcoin price?
AGENT: Based on the current top cryptocurrencies data, Bitcoin's price is $114,607.84. Please note that cryptocurrency prices can fluctuate rapidly, so this is the most recent price available from the market data.

EVALUATION SUMMARY
Pass Rate: 100.0%
Tests Passed: 1/1

DETAILED RESULTS

Test: simple_bitcoin_question
Status: PASSED
Result: All of the expected results can be observed in the conversation.
Reasoning: The agent has provided the current Bitcoin price in USD as requested by the user, fulfilling the expected result. The conversation meets the criteria for category A because